<CENTER>
    <a href="http://opendata.atlas.cern/release/2020/documentation/notebooks/intro.html" class="icons"><img src="../../images/ATLASOD.gif" style="width:40%"></a>
</CENTER>

<CENTER><h1>Searching for the Higgs boson in the H&#8594;&gamma;&gamma; channel (using C++) </h1></CENTER>

## C++ notebook example


**Introduction**
Let's take a current ATLAS Open Data sample and create a histogram:

In order to activate the interactive visualisation of the histogram that is later created we can use the JSROOT magic:

In [ ]:
%jsroot on

We need to include some standard C++ and ROOT libraries

In [2]:
// Creates a TChain to be used by the Analysis.C class
#include <TChain.h>
#include <vector>
#include <TFile.h>
#include <iostream>
#include <string>
#include <stdio.h>

Because we would like to use more than one ROOT input file, the best option is to use a TChain object. This allows to "chain" several samples into a single structure that we can later loop over

In [3]:
//TString path = "https://opendata.cern.ch/eos/opendata/atlas/rucio/opendata/";
TString path = "root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/";

In [4]:
TChain* fChain = new TChain("analysis");

TString prefix = "ODEO_FEB2025_v0";
TString skim = "GamGam";

vector<TString> samples = { "data15_periodD", "data15_periodE", "data15_periodF", "data15_periodG", 
                            "data15_periodH", "data15_periodJ", "data16_periodA", "data16_periodB", 
                            "data16_periodC", "data16_periodD", "data16_periodE", "data16_periodF", 
                            "data16_periodG", "data16_periodI", "data16_periodK", "data16_periodL"
                           };

cout << samples.size() << endl;

int max_samples = 16;

for(int ii=0; ii<max_samples; ii++){
    TString name_sample = samples.at(ii);
    TString name_root_file = path+prefix+"_"+skim+"_"+name_sample+"."+skim+".root";
    cout << name_root_file << endl;
    fChain->Add(name_root_file);
}

int nentries = (Int_t)fChain->GetEntries();
cout << "Number of entries: " << nentries << endl;

16
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodD.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodE.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodF.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodG.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodH.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data15_periodJ.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data16_periodA.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025_v0_GamGam_data16_periodB.GamGam.root
root://eospublic.cern.ch:1094//eos/opendata/atlas/rucio/opendata/ODEO_FEB2025

Now we're going to extract the photons variables

In [5]:
Int_t  photon_n;  //number of preselected photons                                                                                                             

ROOT::VecOps::RVec<float> *photon_pt;  //transverse momentum of the photon                                                                                    
ROOT::VecOps::RVec<float> *photon_eta;  //pseudorapidity of the photon                                                                                        
ROOT::VecOps::RVec<float> *photon_phi;  //azimuthal angle of the photon                                                                                       
ROOT::VecOps::RVec<float> *photon_e;  //energy of the photon                                                                                                  
ROOT::VecOps::RVec<bool> *photon_isTightID;
ROOT::VecOps::RVec<bool> *photon_isTightIso;

Bool_t trigP;
ROOT::VecOps::RVec<float> *photon_ptcone20;

photon_pt = 0;
photon_eta = 0;
photon_phi = 0;
photon_e = 0;
photon_isTightID = 0;
photon_isTightIso = 0;
photon_ptcone20 = 0;

Here we're filling the variables defined above with the content of those inside the input ntuples

In [6]:
fChain->SetBranchAddress("photon_pt",        &photon_pt);
fChain->SetBranchAddress("photon_n",         &photon_n);

fChain->SetBranchAddress("photon_eta",       &photon_eta);
fChain->SetBranchAddress("photon_phi",       &photon_phi);
fChain->SetBranchAddress("photon_e",         &photon_e);
fChain->SetBranchAddress("photon_isTightID", &photon_isTightID);
fChain->SetBranchAddress("photon_isTightIso", &photon_isTightIso);

fChain->SetBranchAddress("trigP",            &trigP);
fChain->SetBranchAddress("photon_ptcone20",  &photon_ptcone20);

We're creating a histogram for this example. The plan in to fill them with events.

In [7]:
//Invariant mass histograms definition
TH1F *h_M_Hyy = new TH1F("h_M_Hyy","Diphoton invariant-mass ; M_{#gamma#gamma} [GeV] ; Events / bin", 30, 105, 160);

We are selecting below a simple look for them.

In [8]:
h_M_Hyy->SetMarkerSize(2.0);
h_M_Hyy->SetLineColor(kBlue);
h_M_Hyy->SetFillColor(kBlue-10);

The Higgs boson analysis implemented here considers Higgs boson decays into a photon pair. The event selection criteria are (this will take a few minutes):

In [9]:
int nbytes = 0;
int accepted_events = 0;
for (int i = 0; i < nentries; i++){
    nbytes = fChain->GetEntry(i);
    if(photon_isTightID->at(0)==true && photon_isTightID->at(1)==true){
        // Cut: pT cut - photon 1 has pT > 50 GeV and photon 2 has pT > 30 GeV
        if(photon_pt->at(0) > 50. && photon_pt->at(1) > 30.){
            // Only the events where the calorimeter isolation is less than 5.5% are kept
            if(photon_ptcone20->at(0)/photon_pt->at(0) < 0.055 && photon_ptcone20->at(1)/photon_pt->at(1) < 0.055){
                // Cut on the pseudorapidity in barrel/end-cap transition region
                if((TMath::Abs(photon_eta->at(0)) < 2.37) && 
                   (TMath::Abs(photon_eta->at(0)) < 1.37 || TMath::Abs(photon_eta->at(0)) > 1.52)){
                    if((TMath::Abs(photon_eta->at(1)) < 2.37) && 
                       (TMath::Abs(photon_eta->at(1)) < 1.37 || TMath::Abs(photon_eta->at(1)) > 1.52)){
                        // TLorentzVector definitions
                        TLorentzVector Photon_1 = TLorentzVector();
                        TLorentzVector Photon_2 = TLorentzVector();

                        Photon_1.SetPtEtaPhiE(photon_pt->at(0), photon_eta->at(0), photon_phi->at(0), photon_e->at(0));
                        Photon_2.SetPtEtaPhiE(photon_pt->at(1), photon_eta->at(1), photon_phi->at(1), photon_e->at(1));
                        
                        // Calculation of the Invariant Mass using TLorentz vectors
                        TLorentzVector Photon_12 = Photon_1 + Photon_2;
                        float inv_mass_Hyy = Photon_12.M();

                        // Cut on null diphoton invariant mass
                        if(inv_mass_Hyy == 0) continue;
                        // Cut on diphoton invariant mass based isolation. Only the events where 
                        // the invididual photon invariant mass based isolation is larger than 35% are kept
                        if((photon_pt->at(0)/inv_mass_Hyy > 0.35) && (photon_pt->at(1)/inv_mass_Hyy > 0.35)){
                            // Filling with the mass of the gamma-gamma system
                            h_M_Hyy->Fill(inv_mass_Hyy);
                            accepted_events+=1;
                        }
                    }
                }
            }
        }
    }
}

// Final message after analysis
std::cout << "* Analysed a total of: " << nentries << " in this sample." << std::endl;
std::cout << "* Number of events passing the selection criteria: " << accepted_events << " in this sample." << std::endl;

* Analysed a total of: 36564144 in this sample.
* Number of events passing the selection criteria: 550057 in this sample.


#### Final plot

In [10]:
TCanvas *cz = new TCanvas("cz", "cz", 900, 600);

// Set up the main pad for the fit plot
TPad *mainPad = new TPad("mainPad", "mainPad", 0, 0.3, 1, 1.0);
mainPad->SetBottomMargin(0);  // Upper and lower pads share the X axis
mainPad->Draw();
mainPad->cd();

double max_h_M_Hyy = h_M_Hyy->GetMaximum();

//h_M_Hyy->SetMaximum(2*max_h_M_Hyy);
h_M_Hyy->SetMarkerStyle(20);  // Data markers
h_M_Hyy->SetMarkerSize(0.8);
h_M_Hyy->SetLineColor(kBlack);
h_M_Hyy->SetStats(0);  // Hide stats box

// Define the signal + background fit (Gaussian for signal + 4th order polynomial for background)
TF1 *sigPlusBkg = new TF1("sigPlusBkg", "gaus(0) + pol4(3)", 105, 160);  // Gaussian + 4th order polynomial
sigPlusBkg->SetParameters(1000, 125, 2);  // Parameters for the Gaussian (amplitude, mean, sigma)
sigPlusBkg->SetLineColor(kRed);
sigPlusBkg->SetLineStyle(1);

// Define the background-only fit (4th-order polynomial)
TF1 *bkgOnly = new TF1("bkgOnly", "pol4", 105, 160);  // 4th order polynomial for background
bkgOnly->SetLineColor(kRed);
bkgOnly->SetLineStyle(2);  // Dashed line for background

// Perform the fit (quiet mode to suppress fit output)
h_M_Hyy->Fit("sigPlusBkg", "RQ");  // Signal + background fit
h_M_Hyy->Fit("bkgOnly", "RQ+");    // Background-only fit

// Draw the data points with error bars
h_M_Hyy->Draw("E1");
sigPlusBkg->Draw("same");  // Draw signal + background fit
bkgOnly->Draw("same");     // Draw background-only fit

// Add legend to main plot
TLegend *legend = new TLegend(0.55, 0.65, 0.9, 0.85);
legend->AddEntry(h_M_Hyy, "Data", "lep");
legend->AddEntry(sigPlusBkg, "Sig+Bkg Fit (m_H = 125 GeV)", "l");
legend->AddEntry(bkgOnly, "Bkg (4th order polynomial)", "l");
legend->Draw();

// Add text labels (ATLAS Open Data, Luminosity, and channel info)
TLatex latex;
latex.SetTextSize(0.04);
latex.DrawLatexNDC(0.2, 0.85, "ATLAS Open Data");
latex.DrawLatexNDC(0.2, 0.8, "#sqrt{s}=13 TeV, #intLdt = 36 fb^{-1}");
latex.DrawLatexNDC(0.2, 0.75, "H #rightarrow #gamma#gamma");

cz->cd();

// Create the residual pad
TPad *residPad = new TPad("residPad", "residPad", 0, 0.05, 1, 0.3);
residPad->SetTopMargin(0);
residPad->SetBottomMargin(0.3);
residPad->Draw();
residPad->cd();

// Calculate residuals (Data - Background)
int nBins = h_M_Hyy->GetNbinsX();
TH1F *residuals = new TH1F("residuals", "", nBins, 105, 160);  // Residuals (data - background)

// Loop over all bins to calculate residuals and plot them
for (int i = 1; i <= nBins; i++) {
    double dataValue = h_M_Hyy->GetBinContent(i);
    double bkgValue = bkgOnly->Eval(h_M_Hyy->GetBinCenter(i));  // Background value at bin center
    double error = h_M_Hyy->GetBinError(i);

    // Residuals = (Data - Background) / Error
    // double residualValue = (dataValue - bkgValue) / error;
    double residualValue = (dataValue - bkgValue);
    
    residuals->SetBinContent(i, residualValue);  // Set residual (Data - Bkg)
}

// Style residual plot components
residuals->SetMarkerStyle(20);  // Data points
residuals->SetMarkerSize(0.8);
residuals->SetLineColor(kBlack);

// Customize axis labels and scales
residuals->GetYaxis()->SetTitle("Events - Bkg");
residuals->GetYaxis()->SetTitleSize(0.07);
residuals->GetYaxis()->SetTitleOffset(0.5);
residuals->GetYaxis()->SetLabelSize(0.07);
residuals->GetXaxis()->SetTitle("di-photon invariant mass m_{#gamma#gamma} [GeV]");
residuals->GetXaxis()->SetTitleSize(0.07);
residuals->GetXaxis()->SetLabelSize(0.07);

// Draw the residuals (Data - Background)
residuals->Draw("E1");

// Create a Gaussian with mean 125, sigma 2 (same as your fit) for the residual plot
TF1 *gaussianResidual = new TF1("gaussianResidual", "gaus(0)", 105, 160);
gaussianResidual->SetParameters(sigPlusBkg->GetParameter(0), sigPlusBkg->GetParameter(1), sigPlusBkg->GetParameter(2));  // Gaussian params: amplitude, mean, sigma
gaussianResidual->SetLineColor(kRed);  // Red color for Gaussian
gaussianResidual->SetLineStyle(1);     // Solid line for Gaussian
gaussianResidual->Draw("same");        // Draw the Gaussian signal on the residual plot

// Draw the dashed red line at Y=0 for the background minus background
TLine *zeroLine = new TLine(105, 0, 160, 0);  // Line from (x=105, y=0) to (x=160, y=0)
zeroLine->SetLineColor(kRed);
zeroLine->SetLineStyle(2);  // Dashed line
zeroLine->Draw("same");

// Display the canvas
cz->Draw();

#### Log Scale

In [11]:
h_M_Hyy->Draw("E3");
cz->SetLogy();
cz->Draw();

**Done!**